# Compare Mannual curated model with the original model

In [ ]:
import cobra
from cobra.io import read_sbml_model, load_json_model
import os
import pandas as pd

pd.set_option("display.max_columns", 20)

print(os.getcwd())

ext_dir = '/../../../data/external'
phenomics = '/../../phenomics'
gapfilling = '/../gapfilling'
mass_balance = '../4.mass_balance'

In [ ]:
model_mass = read_sbml_model(f"{os.getcwd()}/{mass_balance}/Salb-GEM-Biosustain.xml")
model_ori = read_sbml_model(f"{os.getcwd()}/{ext_dir}/Salb-GEM-Updated.xml")

In [ ]:
model_ori

In [ ]:
model_mass

# Calculate activity based on biolog data

In [ ]:
# Mapping of bigg and biolog model
bigg_map_dir = os.getcwd() + ext_dir + "/" + "biolog_bigg_map.csv"
biolog_bigg_map = pd.read_csv(bigg_map_dir)

# Import processed biolog data based on DuctApe result
bio_data_dir = os.getcwd() + phenomics + "/" + 'phenome_J1074.tsv'
bio_data = pd.read_table(bio_data_dir)

In [ ]:
# Join two tables together
bio_data.rename(columns={"#plate_id": "plate_id"}, inplace=True)
bio_data['index'] = bio_data['plate_id'] + "_" + bio_data['well_id']

biolog_bigg_map.rename(columns={"#plate_id": "plate_id"}, inplace=True)
bio_data_merged = bio_data.merge(biolog_bigg_map[['index', 'bigg', 'exchange']], on='index', how='left')

bio_data_merged

In [ ]:
# Find out the samples that grow or not grow based on activity and replicas
# Vote: >=3 = growth, >2/3 replicates = grow

def calculate_agreed_activity(bio_data, activity_cutoff=3, num_agreed_replicas=2):
    """
    Based on the activity data from the microarray experiment, using activity cutoff and
    agreed number replicates cutoff to make a dataframe that contains final activity info.
    
    Input:
    activity_cutoff: an integer between 0-9, if the activity is more than cutoff, it 
    means the strain grows.
    num_agreed_replicas: an integer, if the number of replicates agrees more than
    num_agreed_replicas times, it means the replicas agrees with the experimental result.
    
    Output:
    A dataframe that
    """
    agreed_bio_data = pd.DataFrame(columns=[
    'index', 'activity', 'chemical', 'category', 'moa', 'bigg', 'exchange'])
    
    # for each column 'index'
    for index, group in bio_data.groupby('index'):
        first_row = group.iloc[0]
        if (group['activity'] >= activity_cutoff).sum() >= num_agreed_replicas:
            agreed_bio_data = agreed_bio_data.append({'index': index, 
                                    'activity': True, 
                                    'chemical': first_row['chemical'], 
                                    'category':  first_row['category'], 
                                    'moa':  first_row['moa'],
                                    'bigg': first_row['bigg'],
                                    'exchange': first_row['exchange']}, ignore_index=True)
        else:
            agreed_bio_data = agreed_bio_data.append({'index': index, 
                                    'activity': False, 
                                    'chemical': first_row['chemical'], 
                                    'category':  first_row['category'], 
                                    'moa':  first_row['moa'],
                                    'bigg': first_row['bigg'],
                                    'exchange': first_row['exchange']}, ignore_index=True)
    return agreed_bio_data


activity_cutoff = 3
num_agreed_replicas = 2
agreed_bio_data = calculate_agreed_activity(bio_data_merged, activity_cutoff, num_agreed_replicas)

# Test model simulations functions

In [ ]:
# Define media and model simulation functions
media = {
    "Biolog_base": {  # based on ecocyc
        "EX_cl_e": -1000,
        "EX_na1_e": -1000,
        "EX_nh4_e": -1000,
        "EX_pi_e": -1000,
        "EX_k_e": -1000,
        "EX_so4_e": -1000,
        "EX_mg2_e": -1000,
        "EX_fe2_e": -1000,
        # based on conditions, essentials and trace requirements
        "EX_o2_e": -1000,
        "EX_co2_e": -1000.0,
        "EX_h_e": -1000.0,
        "EX_mn2_e": -1000.0,
        "EX_zn2_e": -1000.0,
        "EX_ca2_e": -1000.0,
        "EX_ni2_e": -1000.0,
        "EX_cu2_e": -1000.0,
        "EX_cobalt2_e": -1000.0,
        "EX_h2o_e": -1000.0,
        "EX_mobd_e": -1000.0,
    },
    "C-Source": {},  # no further modification is needed
    "N-Source": {  # remove ammonia and add C-source
        "EX_nh4_e": 0,
        "EX_pyr_e": -20,
    },
    "P-Source": {  # remove phosphate and add C-source
        "EX_pi_e": 0,
        "EX_pyr_e": -20,
    },
    "S-Source": {  # remove sulfate and add C-source
        "EX_so4_e": 0,
        "EX_pyr_e": -20,
    },
}

def test_model_wrt_biolog(
    moa: str,
    additional_exchange: str,
    model: cobra.core.Model,
    media: dict = media,
    cutoff: float = 0.05,
):
    """
    Tests model with given media and returns True/False based on cutoff growth rate.
    If additional exchange reaction is not in the model, returns NaN.

    Input
    model: metabolic model
    moa: the mechanism of action of the biolog well test
    additional_exchange: the exchange reaction to be added to the model for the well
    media: a dictionary of media conditions to set for a given biolog test
    cutoff: the cutoff growth rate for boolean return

    Output
    True/False based on cutoff growth rate or NaN if additional exchange reaction is not
     in the model
    """
    # drop out quickly if there is no exchange reaction to use in the model
    if additional_exchange not in model.reactions:
        return float("nan")

    with model:
        # reset original media
        for exchange in model.medium.keys():
            model.reactions.get_by_id(exchange).lower_bound = 0

        # set base media
        for exchange, value in media["Biolog_base"].items():
            model.reactions.get_by_id(exchange).lower_bound = value

        for exchange, value in media[moa.split(",")[0]].items():
            model.reactions.get_by_id(exchange).lower_bound = value

        # add exchange reaction. lower bound is high but we are only looking for T/F/NAN
        model.reactions.get_by_id(additional_exchange).lower_bound = -20

        # solve and return
        return True if (model.slim_optimize(error_value=0) >= cutoff) else False

In [ ]:
# Determine the TP, FP, TN and FN

def pred_vs_actual(bio_data, actual, predicted):
    """
    This is a function that summarizes the growth activities by comparing simulated model 
    and actual experimental values.
    
    Input:
    bio_data: A dataframe that contains one column of model siumlation and one column 
    of actual activity.
    actual: A string that is the column name of experimental activity level.
    predicted: A string that is the column name of model value.
    
    Output:
    A 2 by 2 dataframe that that contains the sum of matched True/False values.
    
    """
    return pd.crosstab(bio_data[actual], bio_data[predicted], rownames=['Actual'], colnames=['Predicted'])

# Compare two different models

In [ ]:
# Test original model with cutoff

agreed_bio_data["model_simulation" + "_0.05" + "_model_ori"] = agreed_bio_data.apply(
    func=lambda row: test_model_wrt_biolog(
        moa=row["moa"],
        additional_exchange=row["exchange"],
        model=model_ori,
        media=media,
        cutoff=0.05,
    ),
    axis=1,
)


In [ ]:
agreed_bio_data["model_simulation" + "_0.05"] = agreed_bio_data.apply(
    func=lambda row: test_model_wrt_biolog(
        moa=row["moa"],
        additional_exchange=row["exchange"],
        model=model_mass,
        media=media,
        cutoff=0.05,
    ),
    axis=1,
)

In [ ]:
# cutoff 0.05
confusion_matrix_005_ori = pd.crosstab(agreed_bio_data['activity'], agreed_bio_data['model_simulation_0.05_model_ori'], rownames=['Actual'], colnames=['Predicted'])
print("cutoff 0.05\n", confusion_matrix_005_ori)

confusion_matrix_005_mass = pd.crosstab(agreed_bio_data['activity'], agreed_bio_data['model_simulation_0.05'], rownames=['Actual'], colnames=['Predicted'])
print("cutoff 0.05\n", confusion_matrix_005_mass)

cm_list = [confusion_matrix_005_ori, confusion_matrix_005_mass]

In [ ]:
agreed_bio_data

In [ ]:
import matplotlib.pyplot as plt

# calculate accuracy, precision, sensitivity, specificity, and f-Score for each confusion matrix
# plot all the result into a graph
def calculate_stats(confusion_matrix):
    """
    Description: A function that automatically calculate 5 essential statistics
    
    Input: 
    confusion_matrix: A 2 by 2 dataframe. It's a confusion matrix with row being 
    actual and column being predicted.
    
    Output: 
    result: A dictionary of essential statistics, including accuracy, precision, sensitivity, 
    specificity, f score. 
    """
    result = {}

    result['TN'] = tn = confusion_matrix.iloc[0, 0]
    result['FP'] = fp = confusion_matrix.iloc[0, 1]
    result['FN'] = fn = confusion_matrix.iloc[1, 0]
    result['TP'] = tp = confusion_matrix.iloc[1, 1]

    result['accuracy'] = (tp + tn) / (tp + tn + fp + fn)
    result['precision'] = precision = tp / (tp + fp)
    result['sensitivity'] = sensitivity = tp / (tp + fn)
    result['specificity'] = tn / (tn + fp)
    result['f_score'] = 2 * (precision * sensitivity) / (precision + sensitivity)

    return result


stats_01 = calculate_stats(confusion_matrix_005_ori)
stats_02 = calculate_stats(confusion_matrix_005_mass)

plt.figure(figsize=(10, 4)) 
x = [1, 2]
labels = ['accuracy', 'precision', 'sensitivity', 'specificity', 'f_score']
for i, key in enumerate(labels):
    y = [stats_01[key], stats_02[key]]
    plt.subplot(1, 5, i + 1)
    plt.bar(x, y, width=0.5, color=['tab:red', 'tab:blue'])
    plt.title(key)
    plt.xticks(x, ['Original Model', 'Final Model'], rotation=45, ha="right")

plt.tight_layout()
plt.show()

# Testability of the model

In [ ]:
agreed_bio_data

In [ ]:
# Find out the testable & untestable number of sources from simulation
for element in "CNPS":
    num1 = len(
        agreed_bio_data[
            (agreed_bio_data.xs("moa", axis=1).str.startswith(f"{element}-Source"))
            & ~(agreed_bio_data.xs("model_simulation_0.05_model_ori", axis=1).isna())
        ]
    )
    num2 = len(
        agreed_bio_data[
            (agreed_bio_data.xs("moa", axis=1).str.startswith(f"{element}-Source"))
            & (agreed_bio_data.xs("model_simulation_0.05", axis=1).isna())
        ]
    )
    print(f"{element}-sources: Number of computationally testable: {num1} and untestable: {num2}")

In [ ]:
# Find out the untestable ones
count = 0
for element in "CNPS":
    chemicals = agreed_bio_data[
        (agreed_bio_data.xs("moa", axis=1).str.startswith(f"{element}-Source"))
        & (agreed_bio_data.xs("model_simulation_0.05", axis=1).isna())
        & (agreed_bio_data.xs("activity", axis=1))
    ]["chemical"].unique()
    count += len(chemicals)
    
    print(f"{element}-sources: \nUntestable chemicals with positive growth in experiment: \n{chemicals}\n")
print(count)

# Find out false positives & negatives

In [ ]:
# Find out the inconsistency between predicted and simulated ones
print(f"False positives (actual false, predicted true):\n")
print(agreed_bio_data[
    ~(agreed_bio_data.xs("activity", axis=1))
    & agreed_bio_data.xs("model_simulation_0.05", axis=1)
    ]["chemical"].unique())



In [ ]:
print(f"\nFalse negative (actual true, predicted false):\n")
agreed_bio_data_nan_drop = agreed_bio_data[agreed_bio_data['model_simulation_0.05'].notna()]
false_negative = agreed_bio_data_nan_drop[
    agreed_bio_data_nan_drop.xs("activity", axis=1)
    & ~(agreed_bio_data_nan_drop.xs("model_simulation_0.05", axis=1))
    ]["chemical"].unique()

print(false_negative)

In [ ]:
with model_mass:
    model.medium['EX_gl']

# Compare all the models

In [ ]:

import matplotlib.pyplot as plt
import pandas as pd

# Create a DataFrame with your data
df = pd.read_excel("model_comparison.xlsx")

df

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Convert to long format for seaborn
df_long = pd.melt(df, id_vars=['Model'], 
                 value_vars=['Accuracy', 'Precision', 'Sensitivity', 'F1 Score'],
                 var_name='Metric', value_name='Value')

# Set up a color palette that clearly distinguishes between models
palette = sns.color_palette("viridis", 5)

# Create the plot with seaborn
plt.figure(figsize=(12, 6))
sns.set_style("whitegrid")

# Create the grouped bar plot
ax = sns.barplot(
    data=df_long,
    x="Metric", y="Value", hue="Model",
    palette=palette
)

# Customize the plot
ax.set_title('Model Performance Comparison', fontsize=16)
ax.set_xlabel('')
ax.set_ylabel('Score', fontsize=12)
ax.set_ylim(0.7, 1.05)

# Add value labels on top of bars - compatible with older matplotlib versions
for container in ax.containers:
    bars = container.get_children()
    for bar in bars:
        height = bar.get_height()
        ax.text(
            bar.get_x() + bar.get_width()/2, 
            height + 0.005, 
            f'{height:.3f}', 
            ha='center', 
            va='bottom',
            fontsize=9
        )

# Add a legend with a better position
ax.legend(title='Model', bbox_to_anchor=(0.22, 1))

# Adjust layout
plt.tight_layout()

# Save the figure
plt.savefig("model_performance_comparison.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Plotting
fig, ax = plt.subplots(figsize=(10, 6))

width = 0.2  # width of the bars

# Setting the positions of the bars
positions = list(range(len(df['Model'])))
positions_precision = [x + width for x in positions]
positions_sensitivity = [x + width * 2 for x in positions]
positions_f1 = [x + width * 3 for x in positions]

# Creating the bar plots
bars1 = ax.bar(positions, df['Accuracy'], width, label='Accuracy')
bars2 = ax.bar(positions_precision, df['Precision'], width, label='Precision')
bars3 = ax.bar(positions_sensitivity, df['Sensitivity'], width, label='Sensitivity')
bars4 = ax.bar(positions_f1, df['F1 Score'], width, label='F1 Score')

# Adding labels on top of the bars
for bars in [bars1, bars2, bars3, bars4]:
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), va='bottom', ha='center')

# Adding labels and title
ax.set_xlabel('Model')
ax.set_ylabel('Metrics')
ax.set_title('Model Performance Comparison')
ax.set_xticks([p + width * 1.5 for p in positions])
ax.set_xticklabels(df['Model'])

# Adding a legend
ax.legend()

# Display the plot with guidelines
plt.xticks(rotation=45)
plt.ylim(0.7, 1.05)
plt.show()

In [ ]:
import seaborn as sns 

# Number of models
num_models = len(df)

# Finding the maximum value across all confusion matrices for a consistent color scale
max_value = df[['False Positive', 'True Positive', 'False Negative', 'True Negative']].values.max()

# Creating a subplot for each model's confusion matrix
fig, axes = plt.subplots(1, num_models, figsize=(15,4))

for i, ax in enumerate(axes.flat):
    # Extracting confusion matrix data for each model
    cm_data = [[df.iloc[i]['True Positive'], df.iloc[i]['False Negative']],
               [df.iloc[i]['False Positive'], df.iloc[i]['True Negative']]]
    
    # Creating a heatmap for the confusion matrix with a consistent color scale
    sns.heatmap(cm_data, annot=True, fmt='d', cmap='Blues', vmax=max_value, ax=ax, cbar=False)

    # Adding titles and labels
    ax.set_title(df['Model'][i])
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.xaxis.set_ticklabels(['Positive', 'Negative'])
    ax.yaxis.set_ticklabels(['Positive', 'Negative'])

# Main title for the entire figure
plt.suptitle("Model's Comparison through Confusion Matrix ", fontsize=16, y=1.0)

# Adjust layout
plt.tight_layout()
plt.show()
